In [11]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, SimpleRNN, RNN, LSTM, GRU, SpatialDropout1D, Dropout

In [13]:
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
%matplotlib tk

In [12]:
import os, os.path, sys

In [14]:
path = r"C:\Users\mata2\Desktop\master\podaci\0k\Y"

In [15]:
dir = os.listdir(path)

Ucitavanje fajlova

In [16]:
file_names = []
for file in dir:
    file_names.append(file)

In [17]:
percent_train = 0.8

In [18]:
percent_val = 0.2

In [19]:
train_files_names = file_names[:int(percent_train*len(file_names))]
#test_file_names = file_names[int(percent_train*len(file_names))::]

Ucitavanje zadate koordinate

In [20]:
y_command = np.asarray(pd.read_csv(r"C:\Users\mata2\Desktop\master\podaci\yCmd.txt",header=None).astype('int'))

In [21]:
train_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in train_files_names], axis=0)
# test_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
#                               for file in test_file_names], axis=0)

In [23]:
std_dev = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k Y\Y_std_dev.txt", 'r') as lines:
    for line in lines:
        std_dev.append(float(line.strip('\n')))

In [22]:
sr_vrednost = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k Y\Y_sr_vrednost.txt", 'r') as lines:
    for line in lines:
        sr_vrednost.append(float(line.strip('\n')))

Adekvatno rasporedjivanje podataka kako bi se izbeglo fiktivno stanje

In [24]:
for j in range(len(train_files_names)):
    exec(f"koord_trening_{j} = []")
    exec(f"razlika_trening_{j} = []")

    for i in range(61000):
        exec(f"koord_trening_{j}.append(train_[j*61000+i])")
        exec(f"razlika_trening_{j}.append(y_command[i] - train_[j*61000+i])")
    exec(f'koord_trening_{j} = np.asarray(koord_trening_{j})') 
    exec(f'razlika_trening_{j} = np.asarray(razlika_trening_{j})')  

In [26]:
for j in range(len(train_files_names)):
    exec(f"razlika_trening_{j} = []")
    for i in range(61000):
        exec(f"razlika_trening_{j}.append(y_command[i] - train_[j*61000+i])")
    exec(f'razlika_trening_{j} = np.asarray(razlika_trening_{j})')        

In [25]:
def create_timeseries_split(data, time_steps):

    # Get the number of samples in the data
    num_samples = len(data) - time_steps

    # Create empty arrays to store features and target values
    x_train = np.zeros((num_samples, time_steps, 1))
    y_train = np.zeros((num_samples, 1))

    # Loop through the data and create features and target values
    for i in range(num_samples):
        # Extract a slice of data for the current feature
        x_train[i] = data[i:i+time_steps, :]

        # The target value is the next value after the feature
        y_train[i] = data[i+time_steps, 0]

    return x_train, y_train

In [16]:
min_kord = [float('inf') for x in range(61000)]
max_kord = [float('-inf') for x in range(61000)]

for i in range(61000):    
    for j in range(len(train_files_names)):
        exec(f'current_file = razlika_trening_{j}')
        if current_file[i][0] < min_kord[i]:
            min_kord[i] = current_file[i][0]
        if current_file[i][0] > max_kord[i]:
            max_kord[i] = current_file[i][0]

# TESTIRANJE ARHITEKTURE i LR/BATCH

radjeno u vise iteracija sa razlicitim vrednostima zbog vremenske kompleksnosti

In [27]:
gru_unit = [8,16]
unit1 = [8,16]
unit2 = [8,16]
batch_size = [128,256]
time_steps_list = [8,10]

In [28]:
best_skor = float('-inf')
best_gru = None
best_unit1 = None
best_unit2 = None
best_batch = None
best_step = None
loop = 0
for step in time_steps_list:

    #delimo podatke na osnovu velicine buffera
    input_shape = (step,1)

    for i in range(len(train_files_names)):
        exec(f'train_koord_X_{i}, _ = create_timeseries_split(koord_trening_{i}, step)')
        
    for i in range(len(train_files_names)):
        exec(f'_, train_y_{i} = create_timeseries_split(razlika_trening_{i}, step)')

    train_X = np.asarray(train_koord_X_0)
    for i in range(1,len(train_files_names)):
        exec(f'train_X = np.append(train_X, train_koord_X_{i}, axis=0)')

    train_y = np.asarray(train_y_0)
    for i in range(1,len(train_files_names)):
        exec(f'train_y = np.append(train_y, train_y_{i}, axis=0)')  

    #uzimamo percent_val sa kraja train_X skupa     
    val_x = train_X[-int(len(train_X)*percent_val)-1:-1]
    val_y = train_y[-int(len(train_X)*percent_val)-1:-1]

    train_X = train_X[:len(train_X)-val_x.shape[0]]
    train_y = train_y[:len(train_y)-val_x.shape[0]]
    
    for batch in batch_size:        
        for gru in gru_unit:
            for uni1 in unit1:
                for uni2 in unit2:

                    keras.utils.set_random_seed(7)
                    optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)

                    model = Sequential()
                    model.add(GRU(units = gru, input_shape = input_shape, activation = keras.layers.LeakyReLU()))
                    model.add(Dense(units = uni1, activation=keras.layers.LeakyReLU()))
                    model.add(Dense(units = uni2, activation=keras.layers.LeakyReLU()))
                    model.add(Dense(units = 1, activation=keras.layers.LeakyReLU(), bias_initializer='zeros', kernel_initializer='normal'))
                    model.compile(loss='mean_squared_error', optimizer = optimizerMSE)

                    model.fit(train_X, train_y, epochs = 10, batch_size = batch, verbose = 2, shuffle = False)
                    test_predict_MSE = model.predict(val_x)
                    
                    #zato sto smo uzeli 20% od kraja, ne pocinje indeks od samog pocetka rada masine
                    #radimo evaluaciju na mean 3 sigma testu
                    i =(61000-step) - val_x.shape[0]%(61000-step) -1
                    mean_3std = 0
                    for j in range(len(test_predict_MSE)):
                        if i%61000 == 0:
                            i = step
                        if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
                                mean_3std += 1
                        i += 1
                    
                    loop += 1


                    if mean_3std > best_skor:
                        best_skor = mean_3std
                        best_gru = gru
                        best_unit1 = uni1
                        best_unit2 = uni2
                        best_step = step
                        best_batch = batch

                        print('best_skor: ', best_skor,'best_gru: ', best_gru,'best_unit1: ', best_unit1,'best_unit2: ', best_unit2, 'best_batch', best_batch, 'best_step', best_step)
                    
                    print('loop: ', loop)
print('best_skor: ', best_skor)                    
print('best_gru: ', best_gru)
print('best_unit1: ', best_unit1)
print('best_unit2: ', best_unit2)
print('best_batch: ', best_batch)
print('best_step: ', best_step)


Epoch 1/10

9149/9149 - 31s - loss: 473.5802 - 31s/epoch - 3ms/step
Epoch 2/10
9149/9149 - 31s - loss: 53.1784 - 31s/epoch - 3ms/step
Epoch 3/10
9149/9149 - 36s - loss: 100.4205 - 36s/epoch - 4ms/step
Epoch 4/10
9149/9149 - 36s - loss: 51.7865 - 36s/epoch - 4ms/step
Epoch 5/10
9149/9149 - 36s - loss: 51.3491 - 36s/epoch - 4ms/step
Epoch 6/10
9149/9149 - 36s - loss: 51.7690 - 36s/epoch - 4ms/step
Epoch 7/10
9149/9149 - 35s - loss: 60.9303 - 35s/epoch - 4ms/step
Epoch 8/10
9149/9149 - 34s - loss: 50.5704 - 34s/epoch - 4ms/step
Epoch 9/10
9149/9149 - 37s - loss: 61.3538 - 37s/epoch - 4ms/step
Epoch 10/10
9149/9149 - 37s - loss: 50.0866 - 37s/epoch - 4ms/step
9149/9149 [==============================] - 18s 2ms/step
best_skor:  200212 best_gru:  8 best_unit1:  8 best_unit2:  8 best_batch 128 best_step 8
loop:  1
Epoch 1/10
9149/9149 - 39s - loss: 2780.0847 - 39s/epoch - 4ms/step
Epoch 2/10
9149/9149 - 36s - loss: 57.4666 - 36s/epoch - 4ms/step
Epoch 3/10
9149/9149 - 36s - loss: 55.7210 - 

Epoch 5/10
4575/4575 - 14s - loss: 51.0058 - 14s/epoch - 3ms/step
Epoch 6/10
4575/4575 - 14s - loss: 49.0028 - 14s/epoch - 3ms/step
Epoch 7/10
4575/4575 - 15s - loss: 47.9792 - 15s/epoch - 3ms/step
Epoch 8/10
4575/4575 - 15s - loss: 47.5297 - 15s/epoch - 3ms/step
Epoch 9/10
4575/4575 - 14s - loss: 47.4370 - 14s/epoch - 3ms/step
Epoch 10/10
4575/4575 - 15s - loss: 47.4936 - 15s/epoch - 3ms/step
9149/9149 [==============================] - 13s 1ms/step
loop:  11
Epoch 1/10
4575/4575 - 17s - loss: 60.8166 - 17s/epoch - 4ms/step
Epoch 2/10
4575/4575 - 14s - loss: 59.8095 - 14s/epoch - 3ms/step
Epoch 3/10
4575/4575 - 14s - loss: 62.4781 - 14s/epoch - 3ms/step
Epoch 4/10
4575/4575 - 14s - loss: 87.5149 - 14s/epoch - 3ms/step
Epoch 5/10
4575/4575 - 14s - loss: 59.7986 - 14s/epoch - 3ms/step
Epoch 6/10
4575/4575 - 14s - loss: 61.2371 - 14s/epoch - 3ms/step
Epoch 7/10
4575/4575 - 14s - loss: 61.7898 - 14s/epoch - 3ms/step
Epoch 8/10
4575/4575 - 14s - loss: 60.9610 - 14s/epoch - 3ms/step
Epoch 9

Epoch 6/10
9149/9149 - 35s - loss: 50.5048 - 35s/epoch - 4ms/step
Epoch 7/10
9149/9149 - 35s - loss: 475.2255 - 35s/epoch - 4ms/step
Epoch 8/10
9149/9149 - 35s - loss: 55.3834 - 35s/epoch - 4ms/step
Epoch 9/10
9149/9149 - 33s - loss: 49.2786 - 33s/epoch - 4ms/step
Epoch 10/10
9149/9149 - 34s - loss: 56.8745 - 34s/epoch - 4ms/step
9149/9149 [==============================] - 15s 2ms/step
loop:  22
Epoch 1/10
9149/9149 - 41s - loss: 105.3632 - 41s/epoch - 5ms/step
Epoch 2/10
9149/9149 - 35s - loss: 55.5592 - 35s/epoch - 4ms/step
Epoch 3/10
9149/9149 - 34s - loss: 55.2156 - 34s/epoch - 4ms/step
Epoch 4/10
9149/9149 - 34s - loss: 54.2659 - 34s/epoch - 4ms/step
Epoch 5/10
9149/9149 - 34s - loss: 52.4357 - 34s/epoch - 4ms/step
Epoch 6/10
9149/9149 - 34s - loss: 52.1722 - 34s/epoch - 4ms/step
Epoch 7/10
9149/9149 - 34s - loss: 52.6558 - 34s/epoch - 4ms/step
Epoch 8/10
9149/9149 - 35s - loss: 50.9066 - 35s/epoch - 4ms/step
Epoch 9/10
9149/9149 - 36s - loss: 50.0225 - 36s/epoch - 4ms/step
Epoch